# 拡散モデル: GMM

## シード

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
from torch.optim import Adam


np.random.seed(42) # 乱数生成用のシードを設定
ite = 1 # 乱数生成の回数

random_seed = np.random.randint(0, 10000, ite)  # ランダムな整数値をシード値として取得.例えば 0 〜 9999 の間の整数をite個生成
print("random_seed", random_seed) # 乱数シード値の確認


## GMM学習データ生成

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_uncorrelated_gmm_2d_toy_dataset(num_total_samples=2000,
                                            ratio_mode1=0.5, # モード1のデータが全体の何割を占めるか (0.0 ~ 1.0)
                                            mu1=np.array([-3.0, -3.0]),
                                            sigma1_diag=np.array([1.0, 1.0]), # 対角成分のみ指定
                                            mu2=np.array([3.0, 3.0]),
                                            sigma2_diag=np.array([1.0, 1.0]), # 対角成分のみ指定
                                            random_seed=42):
    """
    2つの無相関な2次元正規分布を組み合わせたガウス混合モデル (GMM) からトイデータセットを生成します。
    各正規分布の比率を指定できます。

    Args:
        num_total_samples (int): 生成する総サンプル数。
        ratio_mode1 (float): モード1 (1つ目の正規分布) のデータが全体の何割を占めるか (0.0 ~ 1.0)。
                              モード2の比率は (1 - ratio_mode1) になります。
        mu1 (np.ndarray): 1つ目の正規分布の平均ベクトル (2次元)。
        sigma1_diag (np.ndarray): 1つ目の正規分布の共分散行列の対角成分 (2要素)。
                                  非対角成分は0と仮定されます。
        mu2 (np.ndarray): 2つ目の正規分布の平均ベクトル (2次元)。
        sigma2_diag (np.ndarray): 2つ目の正規分布の共分散行列の対角成分 (2要素)。
                                  非対角成分は0と仮定されます。
        random_seed (int): 乱数生成のシード。

    Returns:
        np.ndarray: 生成されたデータポイント (N x 2)。
    """
    if not (0.0 <= ratio_mode1 <= 1.0):
        raise ValueError("ratio_mode1 must be between 0.0 and 1.0")

    np.random.seed(random_seed)

    # 各モードのサンプル数を計算
    num_samples_mode1 = int(num_total_samples * ratio_mode1)
    num_samples_mode2 = num_total_samples - num_samples_mode1 # 残りはモード2

    # 共分散行列を作成 (対角成分のみ有効)
    sigma1 = np.diag(sigma1_diag)
    sigma2 = np.diag(sigma2_diag)

    print(f"Generating {num_total_samples} samples:")
    print(f"  Mode 1 ({ratio_mode1*100:.1f}%): {num_samples_mode1} samples")
    print(f"  Mode 2 ({(1-ratio_mode1)*100:.1f}%): {num_samples_mode2} samples")

    # 1つ目の正規分布からデータを生成
    data_mode1 = np.random.multivariate_normal(mu1, sigma1, num_samples_mode1)

    # 2つ目の正規分布からデータを生成
    data_mode2 = np.random.multivariate_normal(mu2, sigma2, num_samples_mode2)

    # 両方のデータを結合
    dataset = np.vstack((data_mode1, data_mode2))

    # データをシャッフル (モードの偏りをなくすため)
    np.random.shuffle(dataset)

    return dataset

def plot_2d_dataset(dataset, title="2D GMM Toy Dataset (Uncorrelated)"):
    """
    2次元データセットをプロットします。

    Args:
        dataset (np.ndarray): プロットするデータセット (N x 2)。
        title (str): グラフのタイトル。
    """
    plt.figure(figsize=(8, 6))
    plt.scatter(dataset[:, 0], dataset[:, 1], s=10, alpha=0.7)
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.axis('equal') # x軸とy軸のスケールを合わせる
    plt.show()

### 生成実行

In [ ]:
# 例4: 比率30:70、円形クラスタ
print("--- Example 4: 30:70 Ratio, Circular Clusters ---")
dataset4 = generate_uncorrelated_gmm_2d_toy_dataset(
    num_total_samples=2000,
    ratio_mode1=0.3,
    mu1=np.array([-3.0, -3.0]),
    sigma1_diag=np.array([1.0, 1.0]),
    mu2=np.array([3.0, 3.0]),
    sigma2_diag=np.array([1.0, 1.0])
)
plot_2d_dataset(dataset4, title="Uncorrelated GMM (30:70, Circular)")

## カーネル密度推定(KDE)

In [ ]:
# カーネル密度推定のための関数
def plot_kde_2d(dataset, title="2D KDE Plot"):
    """
    2次元データセットのカーネル密度推定 (KDE) プロットを作成します。

    Args:
        dataset (np.ndarray): プロットするデータセット (N x 2)。
        title (str): グラフのタイトル。
    """
    from scipy.stats import gaussian_kde

    # KDEを計算
    kde = gaussian_kde(dataset.T)
    x_min, x_max = dataset[:, 0].min(), dataset[:, 0].max()
    y_min, y_max = dataset[:, 1].min(), dataset[:, 1].max()
    
    x_grid = np.linspace(x_min, x_max, 100)
    y_grid = np.linspace(y_min, y_max, 100)
    X, Y = np.meshgrid(x_grid, y_grid)
    Z = kde(np.vstack([X.ravel(), Y.ravel()])).reshape(X.shape)

    # プロット
    plt.figure(figsize=(8, 6))
    plt.contourf(X, Y, Z, levels=20, cmap='viridis')
    plt.colorbar(label='Density')
    plt.scatter(dataset[:, 0], dataset[:, 1], s=10, alpha=0.5, color='white')
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.axis('equal') # x軸とy軸のスケールを合わせる
    plt.show()
# KDEプロットの例
print("--- KDE Plot for Example 4 ---")
plot_kde_2d(dataset4, title="KDE Plot for Uncorrelated GMM (30:70, Circular)")

In [ ]:
print("dataset4.shape: ", dataset4.shape)

## ブートストラッピング(反復抽出)

### 関数の実装

In [ ]:
def bootstrap(data, num_dataset):
    """
    ブートストラップサンプリングを行う関数
    :param data: 元データ
    :param num_dataset: 1ブートストラップサンプルの数
    :param batch_size: 各サンプルのサイズ
    return: ブートストラップサンプルのリスト
    """

    len_data = len(data) # 元データのサイズ
    bootstrap_samples = [] # ブートストラップサンプルを格納する配列
    for i in range(num_dataset):
        indices = np.random.choice(len_data, size=len_data, replace=True) 
        bootstrap_samples.append(data[indices]) # サンプルを格納
    bootstrap_samples = np.array(bootstrap_samples) # numpy配列に変換

    return bootstrap_samples

In [ ]:
print("dataset4.shape: ", dataset4.shape)

### BS実行

In [ ]:
# ブートストラップ法の実行
bootstrap_samples = bootstrap(dataset4, num_dataset=50) # 50個のブートストラップサンプルを生成

# ブートストラップサンプルのプロット
def plot_bootstrap_samples(bootstrap_samples, title="Bootstrap Samples"):
    """
    ブートストラップサンプルをプロットします。

    Args:
        bootstrap_samples (np.ndarray): ブートストラップサンプル (num_samples x D)。
        title (str): グラフのタイトル。
    """
    plt.figure(figsize=(8, 6))
    for i in range(bootstrap_samples.shape[0]):
        plt.scatter(bootstrap_samples[i, :, 0], bootstrap_samples[i, :, 1], s=10, alpha=0.1)
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True)
    plt.axis('equal') # x軸とy軸のスケールを合わせる
    plt.show()

# ブートストラップサンプルのプロット
print("--- Bootstrap Samples Plot ---")
plot_bootstrap_samples(bootstrap_samples, title="Bootstrap Samples from Uncorrelated GMM (30:70, Circular)")

In [ ]:
print("bootstrap_samples.shape: ", bootstrap_samples.shape)

In [ ]:
print("bootstrap_samples[0].shape: ", bootstrap_samples[0].shape)  # 最初のブートストラップサンプルの形状を確認

In [ ]:
# ブートストラップサンプルのkdeプロット
plot_kde_2d(bootstrap_samples.reshape(-1, 2), title="KDE Plot for Bootstrap Samples (30:70, Circular)")

## 生成したデータをKDEにより表示 頂点間距離

In [ ]:
from scipy.stats import gaussian_kde
target_data = bootstrap_samples[0]  # 最初のバッチをターゲットデータとして使用
kde_generated_data = gaussian_kde(target_data.T)

# --- 評価グリッドの定義 ---
# generated_data[i] の範囲に基づいてグリッドを生成
x_min, x_max = target_data[:, 0].min() - 1, target_data[:, 0].max() + 1
y_min, y_max = target_data[:, 1].min() - 1, target_data[:, 1].max() + 1

# グリッドの解像度
num_points = 100
X, Y = np.meshgrid(np.linspace(x_min, x_max, num_points),
                   np.linspace(y_min, y_max, num_points))

# KDEをグリッド上で評価
# (2, num_points*num_points) の形状に変換してkdeに渡す
Z = kde_generated_data(np.vstack([X.ravel(), Y.ravel()]))
Z = Z.reshape(X.shape) # 元のグリッド形状に戻す

# --- 2つの山の頂点を特定 (前回のコードから再利用) ---
# 1. 最初のピーク（グローバル最大値）を見つける
max_z_idx_flat = np.argmax(Z)
row1, col1 = np.unravel_index(max_z_idx_flat, Z.shape)
peak1_coords_2d = np.array([X[row1, col1], Y[row1, col1]])
peak1_density = Z[row1, col1]

# 2. 最初のピーク周辺をマスクして2番目のピークを見つける
Z_masked = Z.copy()
mask_radius = 4 # マスクする半径 (GMMの分散やモード間距離に応じて調整)

distances_from_peak1 = np.sqrt((X - peak1_coords_2d[0])**2 + (Y - peak1_coords_2d[1])**2)
Z_masked[distances_from_peak1 < mask_radius] = -1e10 # 負の大きな値に設定して最大値検索から除外

max_z_masked_idx_flat = np.argmax(Z_masked)
row2, col2 = np.unravel_index(max_z_masked_idx_flat, Z_masked.shape)
peak2_coords_2d = np.array([X[row2, col2], Y[row2, col2]])
peak2_density = Z[row2, col2] # マスク前のZから実際の密度を取得

# --- 3次元空間における距離を計算 ---
# ピークの3D座標を定義 (X, Y, Density)
peak1_coords_3d = np.array([peak1_coords_2d[0], peak1_coords_2d[1], peak1_density])
peak2_coords_3d = np.array([peak2_coords_2d[0], peak2_coords_2d[1], peak2_density])

# 3Dユークリッド距離を計算
distance_3d = np.linalg.norm(peak1_coords_3d - peak2_coords_3d)

print(f"最初のピークの座標: ({peak1_coords_3d[0]:.2f}, {peak1_coords_3d[1]:.2f}, 密度(z): {peak1_coords_3d[2]:.4f})")
print(f"2番目のピークの座標: ({peak2_coords_3d[0]:.2f}, {peak2_coords_3d[1]:.2f}, 密度(z): {peak2_coords_3d[2]:.4f})")
print(f"2D空間におけるピーク頂点間の距離: {np.linalg.norm(peak1_coords_2d - peak2_coords_2d):.2f}")
print(f"3D空間におけるピーク頂点間の距離 (密度をz軸として): {distance_3d:.4f}")

# --- 3Dプロット ---
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# サーフェスプロット
ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.8)

ax.scatter(peak1_coords_3d[0], peak1_coords_3d[1], peak1_coords_3d[2], color='red', s=100, label='Peak 1', zorder=10)
ax.scatter(peak2_coords_3d[0], peak2_coords_3d[1], peak2_coords_3d[2], color='blue', s=100, label='Peak 2', zorder=10)

# ラベルとタイトル
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
ax.set_zlabel('Density')
ax.set_title('3D Kernel Density Estimate from Generated Data')

# 視点の調整 (任意)
ax.view_init(elev=30, azim=150) # 仰角と方位角

# 保存
# plt.savefig('kde_3d_generated_data_plot.png')
# print("KDE_generated_dataに対する3Dの図示を 'kde_3d_generated_data_plot.png' として保存しました。")


In [ ]:
distances = []  # ピーク間の距離を格納するリスト

for i in range(bootstrap_samples.shape[0]):
    target_data = bootstrap_samples[i]  # 最初のバッチをターゲットデータとして使用
    kde_generated_data = gaussian_kde(target_data.T)

    # --- 評価グリッドの定義 ---
    # generated_data[i] の範囲に基づいてグリッドを生成
    x_min, x_max = target_data[:, 0].min() - 1, target_data[:, 0].max() + 1
    y_min, y_max = target_data[:, 1].min() - 1, target_data[:, 1].max() + 1

    # グリッドの解像度
    num_points = 100
    X, Y = np.meshgrid(np.linspace(x_min, x_max, num_points),
                      np.linspace(y_min, y_max, num_points))

    # KDEをグリッド上で評価
    # (2, num_points*num_points) の形状に変換してkdeに渡す
    Z = kde_generated_data(np.vstack([X.ravel(), Y.ravel()]))
    Z = Z.reshape(X.shape) # 元のグリッド形状に戻す

    # --- 2つの山の頂点を特定 (前回のコードから再利用) ---
    # 1. 最初のピーク（グローバル最大値）を見つける
    max_z_idx_flat = np.argmax(Z)
    row1, col1 = np.unravel_index(max_z_idx_flat, Z.shape)
    peak1_coords_2d = np.array([X[row1, col1], Y[row1, col1]])
    peak1_density = Z[row1, col1]

    # 2. 最初のピーク周辺をマスクして2番目のピークを見つける
    Z_masked = Z.copy()
    mask_radius = 4 # マスクする半径 (GMMの分散やモード間距離に応じて調整)

    distances_from_peak1 = np.sqrt((X - peak1_coords_2d[0])**2 + (Y - peak1_coords_2d[1])**2)
    Z_masked[distances_from_peak1 < mask_radius] = -1e10 # 負の大きな値に設定して最大値検索から除外

    max_z_masked_idx_flat = np.argmax(Z_masked)
    row2, col2 = np.unravel_index(max_z_masked_idx_flat, Z_masked.shape)
    peak2_coords_2d = np.array([X[row2, col2], Y[row2, col2]])
    peak2_density = Z[row2, col2] # マスク前のZから実際の密度を取得

    # --- 3次元空間における距離を計算 ---
    # ピークの3D座標を定義 (X, Y, Density)
    peak1_coords_3d = np.array([peak1_coords_2d[0], peak1_coords_2d[1], peak1_density])
    peak2_coords_3d = np.array([peak2_coords_2d[0], peak2_coords_2d[1], peak2_density])

    # 3Dユークリッド距離を計算
    distance_3d = np.linalg.norm(peak1_coords_3d - peak2_coords_3d)

    print(f"最初のピークの座標: ({peak1_coords_3d[0]:.2f}, {peak1_coords_3d[1]:.2f}, 密度(z): {peak1_coords_3d[2]:.4f})")
    print(f"2番目のピークの座標: ({peak2_coords_3d[0]:.2f}, {peak2_coords_3d[1]:.2f}, 密度(z): {peak2_coords_3d[2]:.4f})")
    print(f"2D空間におけるピーク頂点間の距離: {np.linalg.norm(peak1_coords_2d - peak2_coords_2d):.2f}")
    print(f"3D空間におけるピーク頂点間の距離 (密度をz軸として): {distance_3d:.4f}")

    distances.append(distance_3d)  # 距離をリストに追加

    # # --- 3Dプロット ---
    # fig = plt.figure(figsize=(10, 8))
    # ax = fig.add_subplot(111, projection='3d')

    # # サーフェスプロット
    # ax.plot_surface(X, Y, Z, cmap='viridis', edgecolor='none', alpha=0.8)

    # ax.scatter(peak1_coords_3d[0], peak1_coords_3d[1], peak1_coords_3d[2], color='red', s=100, label='Peak 1', zorder=10)
    # ax.scatter(peak2_coords_3d[0], peak2_coords_3d[1], peak2_coords_3d[2], color='blue', s=100, label='Peak 2', zorder=10)

    # # ラベルとタイトル
    # ax.set_xlabel('Dimension 1')
    # ax.set_ylabel('Dimension 2')
    # ax.set_zlabel('Density')
    # ax.set_title('3D Kernel Density Estimate from Generated Data')

    # # 視点の調整 (任意)
    # ax.view_init(elev=30, azim=150) # 仰角と方位角

    # 保存
    # plt.savefig('kde_3d_generated_data_plot.png')
    # print("KDE_generated_dataに対する3Dの図示を 'kde_3d_generated_data_plot.png' として保存しました。")

print("距離算出完了")

In [ ]:
# distancesの平均と標準偏差を計算
mean_distance = np.mean(distances)
std_distance = np.std(distances)
print(f"ブートストラップサンプル間の3D空間におけるピーク頂点間の距離の平均: {mean_distance:.4f}")
print(f"ブートストラップサンプル間の3D空間におけるピーク頂点間の距離の標準偏差: {std_distance:.4f}")

In [ ]:
# 算出した距離の分布
plt.figure(figsize=(8, 6))
plt.hist(distances, bins=30, color='blue', edgecolor='black', alpha=0.7)
plt.title('Distribution of Distances Between Peaks')
plt.xlabel('Distance')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# 確率密度関数から距離の理論値を求める

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

def gaussian_mixture_model_probability_density(x, means, covs, weights):
    """
    多次元ガウス混合モデル (GMM) の確率密度関数を計算します。

    Args:
        x (np.ndarray): 確率密度を評価したいデータポイント。
                        形状は (D,) または (N, D) で、Dはデータの次元数。
        means (list of np.ndarray): 各ガウス成分の平均ベクトルを格納したリスト。
                                    リストの各要素の形状は (D,)。
        covs (list of np.ndarray): 各ガウス成分の共分散行列を格納したリスト。
                                   リストの各要素の形状は (D, D)。
        weights (list or np.ndarray): 各ガウス成分の混合係数（重み）を格納したリスト。
                                      要素の合計は1になる必要があります。

    Returns:
        np.ndarray: 入力データポイント x におけるGMMの確率密度。
                    x が (D,) の場合は float、(N, D) の場合は形状 (N,) のndarray。
    """
    num_components = len(means) # ガウス成分の数
    if not (num_components == len(covs) == len(weights)):
        raise ValueError("means, covs, and weights lists must have the same number of components.")
    if not np.isclose(np.sum(weights), 1.0):
        # 厳密な1.0でなくても、浮動小数点誤差を許容
        print("Warning: Sum of weights is not exactly 1.0. Normalizing weights.")
        weights = np.array(weights) / np.sum(weights)

    # 入力 x が単一の点か、複数の点かによって処理を分ける
    if x.ndim == 1: # 単一のデータポイント (D,)
        total_density = 0.0
        for k in range(num_components):
            # 各ガウス成分の確率密度に重みをかけて加算
            total_density += weights[k] * multivariate_normal.pdf(x, mean=means[k], cov=covs[k])
        return total_density
    elif x.ndim == 2: # 複数のデータポイント (N, D)
        # N個のデータポイントそれぞれに対して密度を計算
        num_data_points = x.shape[0]
        densities = np.zeros(num_data_points)
        for k in range(num_components):
            # 各ガウス成分の確率密度を計算し、重みをかけて加算
            densities += weights[k] * multivariate_normal.pdf(x, mean=means[k], cov=covs[k])
        return densities
    else:
        raise ValueError("Input 'x' must be a 1D (single point) or 2D (multiple points) array.")

# --- 使用例 ---
if __name__ == "__main__":
    # 例1: 2つの2次元ガウス成分を持つGMM
    print("--- 例1: 2つの2次元ガウス成分を持つGMM ---")
    mu1 = np.array([0.0, 0.0])
    sigma1 = np.array([[1.0, 0.5], [0.5, 1.0]]) # 相関あり
    
    mu2 = np.array([5.0, 5.0])
    sigma2 = np.array([[1.5, -0.2], [-0.2, 0.8]]) # 相関あり、形状も異なる

    # 重みは合計が1になるようにする
    weights = [0.4, 0.6] # 成分1が40%、成分2が60%

    means = [mu1, mu2]
    covs = [sigma1, sigma2]

    # 単一の点における確率密度を評価
    point_a = np.array([0.5, 0.5])
    density_a = gaussian_mixture_model_probability_density(point_a, means, covs, weights)
    print(f"点 {point_a} における確率密度: {density_a:.4f}")

    point_b = np.array([4.8, 5.2])
    density_b = gaussian_mixture_model_probability_density(point_b, means, covs, weights)
    print(f"点 {point_b} における確率密度: {density_b:.4f}")

    point_c = np.array([2.5, 2.5]) # モード間の中間点
    density_c = gaussian_mixture_model_probability_density(point_c, means, covs, weights)
    print(f"点 {point_c} における確率密度: {density_c:.4f}")

    # 複数の点における確率密度を評価 (N, D) 形式の入力
    print("\n--- 複数の点における確率密度の評価 ---")
    points_to_evaluate = np.array([
        [0.5, 0.5],
        [4.8, 5.2],
        [2.5, 2.5],
        [-1.0, -1.0],
        [6.0, 6.0]
    ])
    densities = gaussian_mixture_model_probability_density(points_to_evaluate, means, covs, weights)
    for i, point in enumerate(points_to_evaluate):
        print(f"点 {point} における確率密度: {densities[i]:.4f}")

    # 例2: 3つの1次元ガウス成分を持つGMM
    print("\n--- 例2: 3つの1次元ガウス成分を持つGMM ---")
    means_1d = [np.array([-5.0]), np.array([0.0]), np.array([5.0])]
    covs_1d = [np.array([[1.0]]), np.array([[0.5]]), np.array([[2.0]])] # 1次元の場合もcovは(D,D)
    weights_1d = [0.2, 0.5, 0.3]

    point_1d = np.array([0.1])
    density_1d = gaussian_mixture_model_probability_density(point_1d, means_1d, covs_1d, weights_1d)
    print(f"1次元の点 {point_1d} における確率密度: {density_1d:.4f}")

    points_1d_multi = np.array([[-5.5], [-0.2], [4.8], [10.0]])
    densities_1d_multi = gaussian_mixture_model_probability_density(points_1d_multi, means_1d, covs_1d, weights_1d)
    for i, point in enumerate(points_1d_multi):
        print(f"1次元の点 {point} における確率密度: {densities_1d_multi[i]:.4f}")